<a href="https://colab.research.google.com/github/akshay-3apr/UoG-Dissertation/blob/main/Dissertation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Pyterrier

In [ ]:
%pip install -q python-terrier git+https://github.com/cmacdonald/pyterrier_colbert.git faiss-gpu==1.6.3

     |████████████████████████████████| 97 kB 2.8 MB/s 
     |████████████████████████████████| 35.5 MB 203 kB/s 
     |████████████████████████████████| 769 kB 37.4 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 17.9 MB 458 kB/s 
     |████████████████████████████████| 1.2 MB 46.5 MB/s 
     |████████████████████████████████| 880 kB 50.0 MB/s 
     |████████████████████████████████| 3.0 MB 41.4 MB/s 
     |████████████████████████████████| 1.1 MB 37.3 MB/s 
     |████████████████████████████████| 69 kB 3.9 MB/s 
     |████████████████████████████████| 300 kB 60.9 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 45 kB 2.2 MB/s 
     |████████████████████████████████| 6.4 MB 53.0 MB/s 
     |████████████████████████████████| 126 kB 56.8 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 294 kB 60.7 MB/s 
     |████████████████████

## Import pyterrier and initialization

In [ ]:
import pyterrier as pt
import pandas as pd
import sys,gc
import spacy,string,re,nltk
from nltk.stem.porter import *

COLAB='google.colab' in sys.modules

try:
  import faiss
  faiss.get_num_gpus()
except:
  if COLAB:
    print('Installing faiss-gpu from pip ')
    %pip install faiss-gpu==1.6.3
  else:
    print('Installing faiss-gpu via Conda')
    %conda install -c pytorch faiss-gpu

# checks if pyterrier init method is called
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



### Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


### PyTerrier Datasets

In [ ]:
## Get list of dataset
pt.datasets.find_datasets('trec')

[INFO] trec-robust04 is deprecated. Consider using disks45/nocr/trec-robust-2004 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold1 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold1 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold2 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold2 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold3 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold3 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold4 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold4 instead, which provides better parsing of the corpus.
[INFO] trec-robust04/fold5 is deprecated. Consider using disks45/nocr/trec-robust-2004/fold5 instead, which provides better parsing of the corpus.
[INFO] acessing TREC Fair Ranking 2021 through trec-fair-2021 is deprecated; use trec-fair/2021 instead.
[INFO] acessing TREC Fair

,dataset,topics,topics_lang,qrels,corpus,corpus_lang,index,info_url
7,trec-robust-2004,True,en,True,None,None,None,https://trec.nist.gov/data/t13_robust.html
8,trec-robust-2005,True,en,True,None,None,None,https://trec.nist.gov/data/t14_robust.html
9,trec-terabyte,"[2004, 2005, 2006, 2004-2006, 2006-np, 2005-np]",en,"[2004, 2005, 2006, 2004-2006, 2005-np, 2006-np]",None,None,None,https://trec.nist.gov/data/terabyte.html
10,trec-precision-medicine,"[2017, 2018, 2019, 2020]",en,"[qrels-2017-abstracts, qrels-2017-abstracts-sa...",None,None,None,https://trec.nist.gov/data/precmed.html
11,trec-covid,"[round1, round2, round3, round4, round5]",en,"[round1, round2, round3, round3-cumulative, ro...","[round4, round5]",en,True,https://ir.nist.gov/covidSubmit/
...,...,...,...,...,...,...,...,...
612,irds:trec-cast/v1/2019/judged,True,en,True,True,en,None,https://ir-datasets.com/trec-cast.html#trec-ca...
613,irds:trec-cast/v1/2020,"[raw_utterance, automatic_rewritten_utterance,...",en,True,True,en,None,https://ir-datasets.com/trec-cast.html#trec-ca...
614,irds:trec-cast/v1/2020/judged,True,en,True,True,en,None,https://ir-datasets.com/trec-cast.html#trec-ca...
636,trec-deep-learning-docs,"[train, dev, test, test-2020, leaderboard-2020]",en,"[train, dev, test, test-2020]",True,en,True,https://microsoft.github.io/msmarco/


####Importing data

In [ ]:
## Immporting data
## Dataset document: http://data.terrier.org/msmarco_passage.dataset.html
## MS MARCO document: https://microsoft.github.io/msmarco/TREC-Deep-Learning.html
## Get index from PyTerrier dataset

class Dataset:

  index=dataset=collection=test_topics=test_qrels = None

  def importdata(self):
    selecteddataset = int(input("Which dataset to load \n 1. TREC \n 2. MSMARCO-PASSAGE \n Please type the respective number: "))
    if selecteddataset == 1:
      ### TREC DL PASSAGES
      self.dataset = pt.get_dataset('msmarco_passage')
      indexref = self.dataset.get_index(variant='terrier_stemmed')
      self.index = pt.IndexFactory.of(indexref)
      test_topics_19 = self.dataset.get_topics('test-2019')
      test_topics_20 = self.dataset.get_topics('test-2020')
      test_qrels_19 = self.dataset.get_qrels('test-2019')
      test_qrels_20 = self.dataset.get_qrels('test-2020')
      self.test_topics = pd.concat([test_topics_19,test_topics_20]).drop_duplicates()
      self.test_qrels = pd.concat([test_qrels_19,test_qrels_20]).drop_duplicates()
      self.test_topics.qid=self.test_topics.qid.astype("str")
      print("Topics unique count after removing duplicates: ",self.test_topics.qid.unique().size)
      print("Qrels unique count after removing duplicates: ",self.test_qrels.qid.unique().size)

    elif selecteddataset == 2:
      ### MAMARCO PASSAGE
      self.dataset = pt.get_dataset('msmarco_passage')
      indexref = self.dataset.get_index(variant='terrier_stemmed')
      self.index = pt.IndexFactory.of(indexref)
      test_topics_19 = self.dataset.get_topics('test-2019')
      test_topics_20 = self.dataset.get_topics('test-2020')
      test_qrels_19 = self.dataset.get_qrels('test-2019')
      test_qrels_20 = self.dataset.get_qrels('test-2020')
      self.test_topics = pd.concat([test_topics_19,test_topics_20]).drop_duplicates()
      self.test_qrels = pd.concat([test_qrels_19,test_qrels_20]).drop_duplicates()
      self.test_topics.qid=self.test_topics.qid.astype("str")
      print("Topics unique count after removing duplicates: ",self.test_topics.qid.unique().size)
      print("Qrels unique count after removing duplicates: ",self.test_qrels.qid.unique().size)

    return self

  def buildCustomIndex(self,dataframe):
    ### CUSTOM INDEX from DATAFRAME
    pd_indexer = pt.DFIndexer("./index/custom",overwrite=True)
    indexref = pd_indexer.index(dataframe["text"],dataframe["docno"])
    self.index = pt.IndexFactory.of(indexref)
    return self
  
  def getIndex(self):
    return self.index

  def getDataset(self):
    return self.dataset
  
  def setCollection(self):
    ##get collection
    corpus = self.dataset.get_corpus()
    self.collection = pd.read_csv(corpus[0],sep="\t",header=None,names=["docno","text"])
    return self

  def getCollection(self):
    if self.collection is None:
      print("Collection not fetched. Fetch Collection first using 'setCollection()'")
    return self.collection
  
  def getTopics(self):
    return self.test_topics
  
  def getQrels(self):
    return self.test_qrels

  def printCollectionStatistics(self):
    print(self.index.getCollectionStatistics().toString())

### Compare above results
## Both collections are same
# print(trec_collection.docno.unique().size)
# print(msmarco_collection.docno.unique().size)
# collection = pd.concat([trec_collection, msmarco_collection])
# print(collection.shape)
# print(collection.drop_duplicates().shape)
# print(Above count also matched with "https://microsoft.github.io/msmarco/TREC-Deep-Learning-2020")



dataset = Dataset()
dataset.importdata()

Which dataset to load 
 1. TREC 
 2. MSMARCO-PASSAGE 
 Please type the respective number: 2


data.direct.bf:   0%|          | 0.00/486M [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/177M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/377M [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/100M [00:00<?, ?iB/s]

...

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

msmarco-test2020-queries.tsv.gz:   0%|          | 0.00/4.03k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

2020qrels-docs.txt:   0%|          | 0.00/213k [00:00<?, ?iB/s]

Topics unique count after removing duplicates:  400
Qrels unique count after removing duplicates:  97


####Export and Importing MSMARCO pyterrier datatset

In [ ]:
test_qrels = dataset.getQrels()
test_topics = dataset.getTopics()
test_qrels['q']="0"
test_qrels.loc[:,['qid','q','docno', 'label']].to_csv("test_qrels.tsv",sep="\t",header=False,index=False)
!cp -av 'test_qrels.tsv' '/content/mydrive/MyDrive/Dissertation'
test_topics[test_topics.qid.isin(test_qrels.qid)].to_csv("test_topics.tsv",sep="\t",header=False,index=False)
!cp -av 'test_topics.tsv' '/content/mydrive/MyDrive/Dissertation'

'test_qrels.tsv' -> '/content/mydrive/MyDrive/Dissertation/test_qrels.tsv'
'test_topics.tsv' -> '/content/mydrive/MyDrive/Dissertation/test_topics.tsv'


In [ ]:
##importing test_qrels and topics from google drive
test_topics = pd.read_csv('/content/mydrive/MyDrive/Dissertation/test_topics.tsv',sep='\t',header=None,names=['qid','query'],dtype='str')
test_qrels = pd.read_csv('/content/mydrive/MyDrive/Dissertation/test_qrels.tsv',sep='\t',header=None,names=['qid','q','docno', 'label'])

#### Qrels labels relevance:

* [3] Perfectly relevant: The passage is dedicated to the query and contains the exact answer.
* [2] Highly relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden
amongst extraneous information.
* [1] Related: The passage seems related to the query but does not answer it.
* [0] Irrelevant: The passage has nothing to do with the query.





## Helper functions

### Functions to get posting list and raw documents

In [ ]:
## Get document posting list
def getDocuments(index,row):
  docid=row["docid"]
  terms = []
  pointer = index.getDocumentIndex().getDocumentEntry(docid)
  for p in index.getDirectIndex().getPostings(pointer):
    termid = p.getId()
    term = index.getLexicon()[termid].getKey()
    # print(f"\t{term}::{p.getFrequency()}")
    terms.append({term:p.getFrequency()})
  row['documentTerms']=terms
  return row

## Fetch raw documents
def getDocumentsCorpus(corpus,row):
  row['doctext']=corpus[corpus.docno==row.docid].doctext.values[0]
  return row

### Clean up text and get parsed document text

In [ ]:
## Filter records from subgroup top-k
nlp = spacy.load("en_core_web_sm")
stemmer = PorterStemmer()

###
## preprocessing the text
# parameters:
# 1. text: string to be preprocessed
###
def clean_text(text):
    #lower case
    processed_text = text.lower()
    # remove numbers
    processed_text = re.sub(r'\d+','', processed_text)
    # remove punctuations
    processed_text = re.sub(r'[^\w\s]',' ',processed_text)
    #remove special symbols from the text
    processed_text = re.sub(r'[^A-Za-z0-9 ]',' ', processed_text)
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    processed_text = re.sub('\s+', ' ', processed_text).strip()
    return processed_text


###
## filter out top documents
# parameters:
# 1. documents: sorted dataframe over rank to filter out rows
# 2. index: dataset index to fetch the posting list
# 3. topk: value filter out top ranked documents from dataframe
###
def filterTopKRankRecord(documents,index,topK=10):
  documents = documents[documents['rank']<=topK]
  for idx,document in documents.iterrows():
    terms = []
    pointer = index.getDocumentIndex().getDocumentEntry(int(document.docno))
    if pointer is None:
      docid = index.getMetaIndex().getDocument("docno",str(document.docno))
      pointer = index.getDocumentIndex().getDocumentEntry(docid)
    for p in index.getDirectIndex().getPostings(pointer):
      termid = p.getId()
      terms.append(index.getLexicon()[termid].getKey())
    documents.at[idx,'doctext_parsed'] = " ".join(terms)
  return documents

### generate Corpus with IDF score

In [ ]:
## function to calculate corpus and respective IDF for every query

import math

def generateCorpusIDF(documentsdf):
  totaldocument = documentsdf.shape[0]
  corpus = Counter()
  for doc in documentsdf.iterrows():
    for term in set(doc[1].doctext_parsed.split(" ")):
      corpus[term] +=1 
  idf = {}
  for term,termdoccount in corpus.items():
    inversedocfreq = math.log(totaldocument/termdoccount+1)
    idf[term] = inversedocfreq
  return idf



### Building Vocabulary

In [ ]:
###
## building vocabulary 
# parameters:
  # 1. documents: dataframe of documents
  # 2. index: pyTerrier index to retrieve the terms from posting
#
###

def generateVocabulary(documents,index):
  queryCorpus = {}
  for name,group in documents.groupby("qid",as_index=False):
    vocabulary = []
    for idx,doc in group.iterrows():
      pointer = index.getDocumentIndex().getDocumentEntry(int(doc.docno))
      if pointer is None:
        docid = index.getMetaIndex().getDocument("docno",str(doc.docno))
        pointer = index.getDocumentIndex().getDocumentEntry(docid)
      for p in index.getDirectIndex().getPostings(pointer):
        termid = p.getId()
        vocabulary.append(index.getLexicon()[termid].getKey())
    queryCorpus[str(name)] = vocabulary
  
  return queryCorpus

##BM25 Model

In [ ]:
## BM25 retrieval model
bm25 = pt.BatchRetrieve(dataset.getIndex(),num_results=1000, wmodel="BM25")

In [ ]:
retrieved_doc = bm25.transform(dataset.getTopics())
test_qrels['qid']=test_qrels.qid.astype('str')
test_qrels['docno']=test_qrels.docno.astype('str')
retrieved_doc[retrieved_doc.qid=='19335'].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocuments(dataset.getIndex(),row),axis=1)

,qid,docid,docno,rank,score,query,label,q,documentTerms
0,19335,8412684,8412684,0,32.250450,anthropological definition of environment,3,0,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,19335,7267248,7267248,1,28.072633,anthropological definition of environment,0,0,"[{'scientif': 1}, {'develop': 1}, {'2': 1}, {'..."
2,19335,8412687,8412687,2,27.762819,anthropological definition of environment,0,0,"[{'review': 2}, {'innov': 2}, {'human': 2}, {'..."
3,19335,527689,527689,3,27.700537,anthropological definition of environment,0,0,"[{'wai': 1}, {'matter': 2}, {'cultur': 1}, {'b..."
4,19335,3175481,3175481,4,27.698477,anthropological definition of environment,3,0,"[{'scienc': 1}, {'victim': 1}, {'crimin': 1}, ..."
...,...,...,...,...,...,...,...,...,...
134,19335,2594897,2594897,831,15.863715,anthropological definition of environment,0,0,"[{'scienc': 2}, {'see': 1}, {'organ': 1}, {'ef..."
135,19335,3045567,3045567,930,15.506981,anthropological definition of environment,1,0,"[{'scienc': 2}, {'origin': 1}, {'1': 1}, {'rel..."
136,19335,772234,772234,945,15.376898,anthropological definition of environment,0,0,"[{'live': 2}, {'see': 1}, {'develop': 1}, {'we..."
137,19335,6894788,6894788,985,15.361916,anthropological definition of environment,0,0,"[{'understand': 1}, {'caus': 1}, {'present': 2..."


In [ ]:
retrieved_doc.to_csv("BM25_top1000.csv",header=False,index=False,columns=["qid","docid","rank"])
!cp -av 'BM25_top1000.csv' '/content/mydrive/MyDrive/Dissertation'

'BM25_top1000.csv' -> '/content/mydrive/MyDrive/Dissertation/BM25_top1000.csv'


### Query Expansion with RM3

In [ ]:
##Query Expansion with RM3
rm3_pipe = bm25 >> pt.rewrite.RM3(dataset.getIndex(),fb_terms=3,fb_docs=1000) >> bm25
retrieved_doc = rm3_pipe.transform(dataset.getTopics())
retrieved_doc[retrieved_doc.qid=='19335'].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocuments(dataset.getIndex(),row),axis=1)

,qid,docid,docno,rank,score,query_0,query,q,label,documentTerms
0,19335,8412684,8412684,0,29.896002,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,3,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,19335,7267248,7267248,1,26.662123,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'scientif': 1}, {'develop': 1}, {'2': 1}, {'..."
2,19335,527695,527695,2,26.122968,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'origin': 1}, {'who': 1}, {'understand': 1},..."
3,19335,3175481,3175481,3,26.120582,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,3,"[{'scienc': 1}, {'victim': 1}, {'crimin': 1}, ..."
4,19335,527689,527689,4,26.116887,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'wai': 1}, {'matter': 2}, {'cultur': 1}, {'b..."
...,...,...,...,...,...,...,...,...,...,...
120,19335,7763208,7763208,835,14.447151,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'interact': 1}, {'well': 1}, {'peopl': 2}, {..."
121,19335,3045567,3045567,851,14.362389,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,1,"[{'scienc': 2}, {'origin': 1}, {'1': 1}, {'rel..."
122,19335,5078863,5078863,858,14.317698,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'resourc': 1}, {'river': 1}, {'ocean': 1}, {..."
123,19335,429849,429849,998,13.973875,anthropological definition of environment,applypipeline:off environ^0.304093778 definit^...,0,0,"[{'live': 2}, {'scienc': 1}, {'2': 1}, {'group..."


In [ ]:
retrieveddf_2 = rm3_pipe.search("environ^0.304093778 definit^0.295167506 anthropolog^0.400738746")
retrieveddf_2[retrieveddf_2.qid=='1'].merge(test_qrels[test_qrels.qid=='19335'],left_on=['docno'],right_on=['docno'])\
.apply(lambda row:getDocuments(dataset.getIndex(),row),axis=1)

,qid_x,docid,docno,rank,score,query_0,query,qid_y,q,label,documentTerms
0,1,8412684,8412684,0,27.634525,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,3,"[{'attempt': 2}, {'understand': 2}, {'societi'..."
1,1,6999135,6999135,1,27.228236,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'especi': 1}, {'cultur': 3}, {'medic': 1}, {..."
2,1,7267248,7267248,2,27.223818,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'scientif': 1}, {'develop': 1}, {'2': 1}, {'..."
3,1,527695,527695,3,27.112969,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'origin': 1}, {'who': 1}, {'understand': 1},..."
4,1,7298833,7298833,4,26.828246,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'cultur': 5}, {'human': 2}, {'branch': 2}, {..."
...,...,...,...,...,...,...,...,...,...,...,...
116,1,8151642,8151642,774,15.631285,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'live': 1}, {'world': 1}, {'first': 1}, {'na..."
117,1,3045565,3045565,811,15.502663,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,1,"[{'scienc': 2}, {'origin': 2}, {'relationship'..."
118,1,3212083,3212083,851,15.318429,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'scientif': 1}, {'commun': 1}, {'world': 1},..."
119,1,2604487,2604487,874,15.122533,environ^0.304093778 definit^0.295167506 anthro...,applypipeline:off human^0.083801299 environ^0....,19335,0,0,"[{'scienc': 1}, {'includ': 1}, {'approach': 2}..."


##ColBERT Implementation

### Using ColBERT reRank result 

In [ ]:
colbert_res_debasis = pd.read_csv("/content/mydrive/MyDrive/Colab/Dissertation/reranked.res.trec-debasis",sep=" ",header=None,\
            names=["qid","number","docid","rank","scpre","msg"],index_col=False)
colbert_res = pd.read_csv("/content/mydrive/MyDrive/Colab/Dissertation/colbert-ranking.tsv",sep="\t",header=None,\
            names=["qid","docid","rank"],index_col=False)

test_topics_19["qid"]=test_topics_19.qid.astype("int64")

corpus = pt.get_dataset("msmarco_passage").get_corpus()
dfcolbertcor = pd.read_csv(corpus[0],sep="\t",header=None,names=["docno", "doctext"])

colBERT_reranked_data = colbert_res_debasis.merge(test_topics_19,on="qid").apply(lambda row:getDocumentsCorpus(dfcolbertcor,row),axis=1)
colBERT_reranked_data
# colBERT_reranked_data.to_csv("ColBERT_reranked_data.csv",header=False,index=False,columns=["qid","docid","rank","query","doctext"])
# !cp -av 'ColBERT_reranked_data.csv' '/content/mydrive/MyDrive/Colab/Dissertation'

,qid,number,docid,rank,scpre,msg,query,doctext
0,1129237,Q0,8588226,1,27.692362,colbert_reranked,hydrogen is a liquid below what temperature,Hydrogen is a liquid below what temperature? w...
1,1129237,Q0,8588219,2,26.179691,colbert_reranked,hydrogen is a liquid below what temperature,User: Hydrogen is a liquid below what temperat...
2,1129237,Q0,8588222,3,25.720362,colbert_reranked,hydrogen is a liquid below what temperature,Answer to: Hydrogen is a liquid below what tem...
3,1129237,Q0,6080460,4,24.327208,colbert_reranked,hydrogen is a liquid below what temperature,Hydrogen can exist as a liquid under high pres...
4,1129237,Q0,128984,5,24.255947,colbert_reranked,hydrogen is a liquid below what temperature,Hydrogen gas has the molecular formula H 2. At...
...,...,...,...,...,...,...,...,...
42995,915593,Q0,8343631,996,4.192454,colbert_reranked,what types of food can you cook sous vide,"HELPS Word-studies. 2424 IÄsous â IÄsoÃºs,..."
42996,915593,Q0,8343630,997,4.099836,colbert_reranked,what types of food can you cook sous vide,Strong's Concordance. Iesous: IÃ©sous jesus Or...
42997,915593,Q0,5248395,998,4.000244,colbert_reranked,what types of food can you cook sous vide,The Hebrew Bible has yeshuâa; when the Septu...
42998,915593,Q0,2567260,999,3.882811,colbert_reranked,what types of food can you cook sous vide,In a eukaryote cell the process in which the c...


In [ ]:
## An end-to-end ColBERT dense retrieval index can be created as follows:
!rm -rf /content/colbert_indices

from pyterrier_colbert.indexing import ColBERTIndexer
from pyterrier_colbert.ranking import ColBERTFactory
import pandas as pd
import gc


checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"

corpus = pt.get_dataset("msmarco_passage").get_corpus()
dfcolbertcor = pd.read_csv(corpus[0],sep="\t",header=None,names=["docno", "text"])
test_qrels = test_qrels.astype({"docno": int}, errors='raise')
df1 = dfcolbertcor[dfcolbertcor.docno.isin(test_qrels.query("qid=='19335'").docno)]
df2 = dfcolbertcor[~dfcolbertcor.docno.isin(test_qrels.query("qid=='19335'").docno)][:90000]
dfcolbertcor = pd.concat([df2,df1])

## free memory
del df1,df2
gc.collect()

##If indexing is done offline
online_index=True
if online_index:

  indexer = ColBERTIndexer(checkpoint,"/content/colbert_indices","colbert_passage_idx",chunksize=3,ids=True)
  indexer.index(dfcolbertcor.to_dict(orient="records"))
  pyterrier_colbert_factory = indexer.ranking_factory()
  colbert_e2e = pyterrier_colbert_factory.end_to_end()

else:
  ## An end-to-end ColBERT dense retrieval pipeline can be formulated as follows:  

  pytcolbert = ColBERTFactory(checkpoint, "/content/mydrive/MyDrive/Colab/Dissertation/colbert_indices", "colbert_passage_idx")
  colbert_e2e = pytcolbert.end_to_end()

In [ ]:
!ls -ltrh colbert_indices/colbert_passage_idx

total 1.9G
-rw-r--r-- 1 root root 1.3G Jun 29 08:24 0.pt
-rw-r--r-- 1 root root  40M Jun 29 08:24 0.tokenids
-rw-r--r-- 1 root root  63M Jun 29 08:24 0.sample
-rw-r--r-- 1 root root 216K Jun 29 08:24 doclens.0.json
-rw-r--r-- 1 root root 368M Jun 29 08:28 1.pt
-rw-r--r-- 1 root root  12M Jun 29 08:28 1.tokenids
-rw-r--r-- 1 root root  19M Jun 29 08:28 1.sample
-rw-r--r-- 1 root root  63K Jun 29 08:28 doclens.1.json
-rw-r--r-- 1 root root 173K Jun 29 08:28 docnos.pkl.gz
-rw-r--r-- 1 root root 153M Jun 29 08:31 ivfpq.100.faiss


In [ ]:
## Copy indexed folder to drive for backup
!tar -zcvf colbert_indices/colbert_passage_idx.tar.gz colbert_indices/colbert_passage_idx
!cp -av 'colbert_indices/colbert_passage_idx.tar.gz' '/content/mydrive/MyDrive/Colab/Dissertation'

!tar -xzvf "/content/mydrive/MyDrive/Colab/Dissertation/colbert_passage_idx.tar.gz" -C "/content/mydrive/MyDrive/Colab/Dissertation/"

!ls -ltr /content/mydrive/MyDrive/Colab/Dissertation/colbert_indices/colbert_passage_idx/

colbert_indices/colbert_passage_idx/
colbert_indices/colbert_passage_idx/doclens.0.json
colbert_indices/colbert_passage_idx/doclens.1.json
colbert_indices/colbert_passage_idx/ivfpq.100.faiss
colbert_indices/colbert_passage_idx/1.pt
colbert_indices/colbert_passage_idx/0.tokenids
colbert_indices/colbert_passage_idx/1.sample
colbert_indices/colbert_passage_idx/docnos.pkl.gz
colbert_indices/colbert_passage_idx/0.sample
colbert_indices/colbert_passage_idx/0.pt
colbert_indices/colbert_passage_idx/1.tokenids
'colbert_indices/colbert_passage_idx.tar.gz' -> '/content/mydrive/MyDrive/Colab/Dissertation/colbert_passage_idx.tar.gz'
colbert_indices/colbert_passage_idx/
colbert_indices/colbert_passage_idx/doclens.0.json
colbert_indices/colbert_passage_idx/doclens.1.json
colbert_indices/colbert_passage_idx/ivfpq.100.faiss
colbert_indices/colbert_passage_idx/1.pt
colbert_indices/colbert_passage_idx/0.tokenids
colbert_indices/colbert_passage_idx/1.sample
colbert_indices/colbert_passage_idx/docnos.pkl.g

In [ ]:
retrieved_doc = colbert_e2e.transform(dataset.get_topics('test-2019'))

# retrieved_doc = colbert_e2e.search('anthropological definition of environment')

test_qrels = test_qrels.astype({"docno": int}, errors='raise')

retrieved_doc[retrieved_doc.qid=="19335"].merge(test_qrels,left_on=['qid','docno'],right_on=['qid','docno'])\
.apply(lambda row:getDocument(dfcolbertcor,row),axis=1).sort_values(by=['rank'])

###Query Expansion with RM3 for ColBERT

## Run Experiment

In [ ]:
def retExperiment(retr_systems,topics,qrels,metrics=['map'],dataframe=True,baseline=None,names=None,perquery=False,verbose=True):
  return pt.Experiment(
      retr_systems=retr_systems,
      topics = topics,
      qrels = qrels,
      eval_metrics=metrics,
      round=4,
      dataframe=dataframe,
      baseline=baseline,
      names = names,
      perquery = perquery,
      verbose=verbose
)
retExperiment(retr_systems = [bm25,rm3_pipe],topics=test_topics,qrels=test_qrels,baseline=0,names=["bm25","rm3"])

## ColBERT and BM25 comparison

### Check the similarity between BM25 with Query Expansion and ColBERT

In [ ]:
### Reranked dataset in my own loptop using Standford ColBERT implementation
### Github link: https://github.com/stanford-futuredata/ColBERT/blob/master/README.md

colbert_reranked_docs = pd.read_csv('/content/mydrive/MyDrive/Dissertation/ColBERT_reranked_data.csv',
                                    names=["qid","docid","rank","query","doctext"])
colbert_docs_50 = colbert_reranked_docs.groupby("qid",as_index=False).apply(lambda subgroup:filterTopKRankRecord(subgroup,dataset.getIndex(),topK=50))

In [ ]:
### Debasis TREC DL reranked output
colbert_res = pd.read_csv("/content/mydrive/MyDrive/Dissertation/TREC-DL-data/reranked.res.trec-debasis",sep=" ",header=None,\
            names=["qid","number","docno","rank","score","msg"],index_col=False)
colbert_res.sort_values(["qid","rank"],inplace=True)
colbert_res.qid=colbert_res.qid.astype('str')
colbert_res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97000 entries, 56000 to 17999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   qid     97000 non-null  object 
 1   number  97000 non-null  object 
 2   docno   97000 non-null  int64  
 3   rank    97000 non-null  int64  
 4   score   97000 non-null  float64
 5   msg     97000 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 5.2+ MB


### Getting Terms weights using Jaccard Similarity

In [ ]:
#build custom index for only colBERT reranked output
collection = dataset.setCollection().getCollection()
colbert_res = colbert_res.merge(collection,left_on="docno",right_on="docno")
indexDataset = colbert_res.loc[:,['text','docno']]
indexDataset.docno = indexDataset.docno.astype('str')
index = dataset.buildCustomIndex(indexDataset).getIndex()

collection.tar.gz:   0%|          | 0.00/987M [00:00<?, ?iB/s]

00:45:42.424 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (799603) - further warnings are suppressed
00:46:25.538 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents


In [ ]:
## code to fetch terms from corpus and check the jaccard simiarity between the BM25 with expanded query and ColBERT
import os
from tqdm import tqdm

def calTermWeights(queryVocabularies,basemodel,colbert_docs,queries,similarity_type):
  # changing dtype of colbert_docs docno column for comparison
  colbert_docs.docno = colbert_docs.docno.astype('str')
  retrieved_doc = basemodel.transform(queries)
  # print(retrieved_doc.info())
  basesimscore = similarity(retrieved_doc,colbert_docs)
  jaccsimilarity = []
  for row in tqdm(queries.to_dict(orient="records")):
    qid,query = row['qid'],row['query']
    queryVocab = queryVocabularies[qid]

    ##Improvement###
    words = [word for word in queryVocab if not word in query.split(" ")]
    expanded_query = [(count,query+" "+word) for count,word in enumerate(words,start=1)]
    retrieved_doc = basemodel.transform(pd.DataFrame(expanded_query,columns=["qid","query"]))
    retrieved_doc.qid=qid
    # print("\n",retrieved_doc.info())
    for idx,group in retrieved_doc.groupby("query",as_index=False):
      word = group['query'].unique()[0].split(" ")[-1]
      jscore = similarity(group,colbert_docs,type=similarity_type)
      diff = checkIncrement(basesimscore,jscore)
      jaccsimilarity.append((qid,query,word,jscore,diff))
  return jaccsimilarity

#function to calculate similarity measure
def similarity(bm25,colbert,type="jaccard"):
  score = -99
  if type.lower() == "jaccard":
    for qid in bm25.qid.unique():
      bm25querysubset = set(bm25[bm25.qid==qid]['docno'])
      colbertquerysubset = set(colbert[colbert.qid==qid]['docno'])
      jaccardscore = float(len(bm25querysubset.intersection(colbertquerysubset))) / len(bm25querysubset.union(colbertquerysubset))
      score=jaccardscore
  else:
    print(f"{type} similarity yet to be implemented.")
  return score

#function to calculate improvement
def checkIncrement(baseJaccardScore,currentJaccardScore):
  #check relative increment in the jaccard similarity score for each term
  return float((currentJaccardScore-baseJaccardScore)/baseJaccardScore)

## get baseline model
bm25 = pt.BatchRetrieve(dataset.getIndex(),num_results=50, wmodel="BM25")

## get topK of ColBERT reranked output
colbert_res = colbert_res.groupby("qid",as_index=False).apply(lambda subgroup:filterTopKRankRecord(subgroup,dataset.getIndex(),topK=50))
##filter out test_topics
queries = test_topics[test_topics.qid.isin(colbert_res.qid)]


#initiate the process
queryVocabulary = generateVocabulary(colbert_res,index=dataset.getIndex())
result = calTermWeights(queryVocabulary,bm25,colbert_res,queries,'jaccard')
termWeights = pd.DataFrame(result, columns =['qid','original_query','word','jscore','improvement'])
termWeights.to_csv("ColBERT_BM25_termweights_trec_dl.csv",header=['qid','original_query','word','jscore','improvement'],index=False)
!cp -av 'ColBERT_BM25_termweights_trec_dl.csv' '/content/mydrive/MyDrive/Dissertation'

100%|██████████| 97/97 [06:47<00:00,  4.20s/it]


'ColBERT_BM25_termweights_trec_dl.csv' -> '/content/mydrive/MyDrive/Dissertation/ColBERT_BM25_termweights_trec_dl.csv'


### Using term Weights get the average query performance

In [ ]:
## fetch features based on jaccard similarity
def fetchFeature(group,rowcount=3):
  group = group.sort_values("improvement",ascending=False).head(rowcount)
  group["expanded_terms"] = group['word'].str.cat(sep=' ')
  group["final_query"] = group["original_query"]+" "+group['expanded_terms']
  return group

### read the csv of term weights
termWeights = pd.read_csv('/content/mydrive/MyDrive/Dissertation/ColBERT_BM25_termweights_trec_dl.csv')
termWeights = termWeights[~termWeights.word.isin(termWeights.original_query.unique()[0].split(' '))]
queries = termWeights.groupby('qid',as_index=False).apply(lambda group:fetchFeature(group,rowcount=10)).reset_index()
queries.drop(['level_0','level_1'],axis=1,inplace=True)
queries = queries.loc[:,['qid','original_query','expanded_terms','final_query']].groupby('qid',as_index=False).first()
queries.qid=queries.qid.astype('str')

##filter out test_topics
test_topics = test_topics[test_topics.qid.isin(colbert_res.qid)]

## get baseline performance
bm25 = pt.BatchRetrieve(index,num_results=50, wmodel="BM25")
retrieved_doc = bm25.transform(test_topics)
base = similarity(retrieved_doc,colbert_res)
jaccsimilarity = []

for idx,data in tqdm(queries.iterrows()):
  retrieved_doc = bm25.search(data.final_query)
  retrieved_doc.qid = data.qid
  jscore = similarity(retrieved_doc,colbert_res,type='jaccard')
  diff = checkIncrement(base,jscore)
  jaccsimilarity.append((data.qid,data.original_query,data.expanded_terms,jscore,diff))


df = pd.DataFrame(jaccsimilarity, columns =['qid','original_query','word','jscore','improvement'])
df.to_csv("ColBERT_BM25_jaccard_trec_dl.csv",header=['qid','original_query','word','jscore','improvement'],index=False)
!cp -av 'ColBERT_BM25_jaccard_trec_dl.csv' '/content/mydrive/MyDrive/Dissertation'

97it [00:01, 67.50it/s]


'ColBERT_BM25_jaccard_trec_dl.csv' -> '/content/mydrive/MyDrive/Dissertation/ColBERT_BM25_jaccard_trec_dl.csv'


In [ ]:
### Avg jaccard score
# df = pd.read_csv('/content/mydrive/MyDrive/Dissertation/ColBERT_BM25_jaccard_trec_dl_2019.csv')
print("Average Jaccard: ",df.jscore.mean(),"\n")
df

Average Jaccard:  0.37549099141333153 



,qid,original_query,word,jscore,improvement
0,19335,anthropological definition of environment,anthropocentr laej citya milieu sardinia redfi...,0.351351,0.479374
1,23849,are naturalization records public information,open kept unaffili extern freedom access conte...,0.219512,-0.075738
2,42255,average salary for dental hygienist in nebraska,typic hourli yearli statist start labor bureau...,0.219512,-0.075738
3,47210,average wedding dress alteration cost,bodic bead gown hem 200 bottom bridal simpl sh...,0.428571,0.804511
4,47923,axon terminals or synaptic knob definition,gland slender ge effector neral interneuron co...,0.449275,0.891686
...,...,...,...,...,...
92,1133167,how is the weather in jamaica,tropic cooler winter higher hot climat region ...,0.333333,0.403509
93,1133579,how does granulation tissue start,rich fill place site fade form time tri new dim,0.388889,0.637427
94,1136043,difference between a hotel and motel,motor portmanteau fog join smog blend brunch c...,0.302632,0.274238
95,1136047,difference between a company s strategy and bu...,achiev strengthen tactic competit goal incom i...,0.306667,0.291228
